In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
column_names = [
    'Status_checking_account', 'Duration', 'Credit_history', 'Purpose', 'Credit_amount',
    'Savings_account_bonds', 'Employment', 'Installment_rate', 'Personal_status_and_sex',
    'Other_debtors', 'Residence_since', 'Property', 'Age', 'Other_installment_plans',
    'Housing', 'Number_credits', 'Job', 'People_liable', 'Telephone', 'Foreign_worker', 'Risk'
]

df = pd.read_csv("german.data", sep=' ', header=None, names=column_names)


df['target'] = df['Risk'].map({1: 0, 2: 1})
df.drop(columns=['Risk'], inplace=True)

categorical_cols = [
    'Status_checking_account', 'Credit_history', 'Purpose', 'Savings_account_bonds',
    'Employment', 'Personal_status_and_sex', 'Other_debtors', 'Property',
    'Other_installment_plans', 'Housing', 'Job', 'Telephone', 'Foreign_worker'
]
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['target']]

for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
X = df.drop(columns=['target'])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

def evaluate_model(model, X_test, y_test):
    y_pred_prob = model.predict(X_test).ravel()
    y_pred = (y_pred_prob > 0.5).astype(int)
    print("✅ Accuracy:", accuracy_score(y_test, y_pred))
    print("📈 F1 Score:", f1_score(y_test, y_pred))
    print("🏆 ROC AUC:", roc_auc_score(y_test, y_pred_prob))
    print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


In [3]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.2, epochs=70, batch_size=32, verbose=1)



Epoch 1/70


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5829 - loss: 0.7265 - val_accuracy: 0.6687 - val_loss: 0.6322
Epoch 2/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7058 - loss: 0.5745 - val_accuracy: 0.6750 - val_loss: 0.5664
Epoch 3/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6984 - loss: 0.5571 - val_accuracy: 0.6938 - val_loss: 0.5566
Epoch 4/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7579 - loss: 0.4877 - val_accuracy: 0.7063 - val_loss: 0.5311
Epoch 5/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7736 - loss: 0.5177 - val_accuracy: 0.7188 - val_loss: 0.5383
Epoch 6/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7205 - loss: 0.5278 - val_accuracy: 0.6938 - val_loss: 0.5279
Epoch 7/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7708 - loss: 0.5018 - val_accuracy: 0.7125 - val_loss: 0.5320
Epoch 8/70
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7354 - loss: 0.5138 - val_accuracy: 0.7125 - val_loss: 0.5217
Ep

In [4]:

evaluate_model(model, X_test, y_test)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
✅ Accuracy: 0.785
📈 F1 Score: 0.6386554621848739
🏆 ROC AUC: 0.7854761904761904

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.85       140
           1       0.64      0.63      0.64        60

    accuracy                           0.79       200
   macro avg       0.74      0.74      0.74       200
weighted avg       0.78      0.79      0.78       200

